# RAG using the saved weviate embeddings
Here we will use Retrival Augumented Generation using BM25 and cross encoder re-ranking

In [1]:
import weaviate
from datetime import datetime, timezone

# Connect to Weaviate
client = weaviate.connect_to_local(host="localhost", port=8080)
print(f"Connected to Weaviate: {client.is_ready()}")

Connected to Weaviate: True


In [2]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
azure_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Use deployment name (what you created in Azure Portal)
embedding_deployment = "text-embedding-3-large"

print("Azure OpenAI client initialized successfully")
print(f"Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"Using deployment: {embedding_deployment}")


Azure OpenAI client initialized successfully
Endpoint: https://grow-me82mm7z-eastus2.services.ai.azure.com
Using deployment: text-embedding-3-large


In [3]:
def vector_similarity_search(query: str, top_k: int = 5):
    """
    Perform vector similarity search using Azure OpenAI embeddings.
    
    Args:
        query: Search query string
        top_k: Number of top results to return
        
    Returns:
        List of results with scores and metadata
    """
    # Generate embedding for the query
    response = azure_client.embeddings.create(
        input=query,
        model=embedding_deployment
    )
    query_embedding = response.data[0].embedding
    
    # Get Document collection
    documents = client.collections.get("Document")
    
    # Perform vector similarity search with distance metric
    results = documents.query.near_vector(
        near_vector=query_embedding,
        limit=top_k,
        distance=0.7,  # Return results within cosine distance 0.7
        return_metadata=weaviate.classes.query.MetadataQuery(
            distance=True  # Get actual distance instead of certainty
        )
    ).objects
    
    # Format results
    formatted_results = []
    for obj in results:
        # Distance: lower is better (0 = identical, 2 = opposite)
        # Convert to similarity score: 1 - distance for easier interpretation
        distance = obj.metadata.distance if obj.metadata else 0
        similarity_score = 1 - distance if distance else 0
        
        formatted_results.append({
            "score": similarity_score,
            "distance": distance,
            "content": obj.properties.get("content", ""),
            "contextualized_chunk": obj.properties.get("contextualized_chunk", ""),
            "source": obj.properties.get("source", ""),
            "page_number": obj.properties.get("page_number"),
            "chunk_type": obj.properties.get("chunk_type", ""),
            "breadcrumb": obj.properties.get("breadcrumb", "")
        })
    
    return formatted_results

print("Vector similarity search function created ✓")


Vector similarity search function created ✓


In [4]:
def bm25_search(query: str, top_k: int = 5):
    """
    Perform BM25 keyword search on contextualized chunks.
    
    Args:
        query: Search query string
        top_k: Number of top results to return
        
    Returns:
        List of results with BM25 scores and metadata
    """
    # Get Document collection
    documents = client.collections.get("Document")
    
    # Perform BM25 search on contextualized_chunk property only
    results = documents.query.bm25(
        query=query,
        query_properties=["contextualized_chunk"],  # Search only contextualized_chunk
        limit=top_k,
        return_metadata=weaviate.classes.query.MetadataQuery(
            score=True,
            explain_score=True
        )
    ).objects
    
    # Format results
    formatted_results = []
    for obj in results:
        formatted_results.append({
            "score": obj.metadata.score if obj.metadata else None,
            "content": obj.properties.get("content", ""),
            "contextualized_chunk": obj.properties.get("contextualized_chunk", ""),
            "source": obj.properties.get("source", ""),
            "page_number": obj.properties.get("page_number"),
            "chunk_type": obj.properties.get("chunk_type", ""),
            "breadcrumb": obj.properties.get("breadcrumb", "")
        })
    
    return formatted_results

print("BM25 search function created ✓")


BM25 search function created ✓


In [5]:
def display_results(vector_results, bm25_results, query):
    """
    Display and compare vector search vs BM25 search results.
    
    Args:
        vector_results: Results from vector similarity search
        bm25_results: Results from BM25 search
        query: The original query
    """
    print("\n" + "="*80)
    print(f"QUERY: {query}")
    print("="*80)
    
    # Vector Search Results
    print("\n--- VECTOR SIMILARITY SEARCH (Top Results) ---")
    for i, result in enumerate(vector_results, 1):
        sim_score = result['score']
        dist = result.get('distance', 0)
        print(f"\n{i}. [Similarity: {sim_score:.6f} | Distance: {dist:.6f}]")
        print(f"   Source: {result['source']} | Page: {result['page_number']}")
        print(f"   Type: {result['chunk_type']}")
        if result['breadcrumb']:
            print(f"   Breadcrumb: {result['breadcrumb']}")
        print(f"   Content: {result['content'][:200]}..." if len(result['content']) > 200 else f"   Content: {result['content']}")
    
    # BM25 Search Results
    print("\n\n--- BM25 KEYWORD SEARCH (Top Results) ---")
    for i, result in enumerate(bm25_results, 1):
        print(f"\n{i}. [Score: {result['score']:.4f}]")
        print(f"   Source: {result['source']} | Page: {result['page_number']}")
        print(f"   Type: {result['chunk_type']}")
        if result['breadcrumb']:
            print(f"   Breadcrumb: {result['breadcrumb']}")
        print(f"   Content: {result['content'][:200]}..." if len(result['content']) > 200 else f"   Content: {result['content']}")
    
    print("\n" + "="*80)

print("Display function created ✓")


Display function created ✓


## Comparison: Vector Search vs BM25 Search

Run test queries to compare the two retrieval methods below.


In [6]:
# Example Query 2 - Try your own query
query2 = "government structure and executive powers"

print("\n🔍 Running retrieval comparison for Query 2...")
vector_results_2 = vector_similarity_search(query2, top_k=5)
bm25_results_2 = bm25_search(query2, top_k=5)
display_results(vector_results_2, bm25_results_2, query2)



🔍 Running retrieval comparison for Query 2...

QUERY: government structure and executive powers

--- VECTOR SIMILARITY SEARCH (Top Results) ---

1. [Similarity: 0.470117 | Distance: 0.529883]
   Source: nepal_constitution.pdf | Page: 85
   Type: section
   Breadcrumb: Part 16 Financial Procedures of Province > Article 213; Part 17 Local Executive > Article 214, 215
   Content: (3) The amount of the expenditures so m et shall be reimbursed as soon as possible as per the Provincial Act.213.Act relating to financial procedures : Matters relating to the transfer of money approp...

2. [Similarity: 0.457522 | Distance: 0.542478]
   Source: nepal_constitution.pdf | Page: 69
   Type: section
   Breadcrumb: PART 13 Provincial Executive
   Content: (4) The provincial executive power, pursuant to this Constitution, shall be as provided in the list of competencies of the Province in Schedule -6 and the list of Concurrent competencies/powers in Sch...

3. [Similarity: 0.450921 | Distance: 0.54907

In [7]:
def reciprocal_rank_fusion(bm25_results, vector_results, k=60):
    """
    Combine BM25 and vector results using Reciprocal Rank Fusion.
    
    RRF Score: sum(1 / (k + rank)) for each method
    
    Args:
        bm25_results: List of BM25 results (ranked)
        vector_results: List of vector results (ranked)
        k: Constant for RRF (default 60)
        
    Returns:
        Combined results sorted by RRF score
    """
    rrf_scores = {}
    result_map = {}
    
    # Process BM25 results
    for rank, result in enumerate(bm25_results, 1):
        doc_id = result['content'][:100]
        rrf_score = 1 / (k + rank)
        
        if doc_id not in rrf_scores:
            rrf_scores[doc_id] = 0
            result['source_method'] = 'BM25'
            result['original_rank_bm25'] = rank
            result['bm25_score'] = result.get('score')
            result_map[doc_id] = result
        
        rrf_scores[doc_id] += rrf_score
    
    # Process vector results (skip if already in BM25)
    bm25_ids = {r['content'][:100] for r in bm25_results}
    for rank, result in enumerate(vector_results, 1):
        doc_id = result['content'][:100]
        
        if doc_id not in bm25_ids:
            rrf_score = 1 / (k + rank)
            rrf_scores[doc_id] = rrf_score
            result['source_method'] = 'Vector'
            result['original_rank_vector'] = rank
            result['vector_score'] = result.get('score')
            result_map[doc_id] = result
    
    # Assign RRF scores and sort
    for doc_id, result in result_map.items():
        result['rrf_score'] = rrf_scores[doc_id]
    
    combined = sorted(result_map.values(), key=lambda x: x['rrf_score'], reverse=True)
    return combined

print("Reciprocal Rank Fusion function created")


Reciprocal Rank Fusion function created


In [8]:
from sentence_transformers import CrossEncoder

cross_encoder_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")
print("Cross-encoder model loaded")

def cross_encoder_rerank(query, candidates, top_k=10):
    """
    Re-rank candidates using cross-encoder model.
    
    Args:
        query: Search query
        candidates: List of candidate results
        top_k: Number of top results to return
        
    Returns:
        Re-ranked results sorted by cross-encoder score
    """
    candidate_texts = [c['content'] for c in candidates]
    pairs = [[query, text] for text in candidate_texts]
    
    scores = cross_encoder_model.predict(pairs)
    
    for candidate, score in zip(candidates, scores):
        candidate['cross_encoder_score'] = float(score)
    
    ranked = sorted(candidates, key=lambda x: x['cross_encoder_score'], reverse=True)
    return ranked[:top_k]

print("Cross-encoder re-ranking function created")


/home/epein5/Desktop/ADVANCED-RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cross-encoder model loaded
Cross-encoder re-ranking function created


## Hybrid Retrieval: RRF + Cross-Encoder Re-ranking

Combines BM25 and vector search using Reciprocal Rank Fusion, then re-ranks with a cross-encoder model.


In [9]:
query = "What are the fundamental rights and freedoms?"

# Step 1: Get results from both methods
bm25_results = bm25_search(query, top_k=25)
vector_results_all = vector_similarity_search(query, top_k=50)

# Filter vector results to get unique ones
bm25_ids = {r['content'][:100] for r in bm25_results}
vector_unique = [r for r in vector_results_all if r['content'][:100] not in bm25_ids][:25]

# Step 2: Combine with RRF and get final results
rrf_combined = reciprocal_rank_fusion(bm25_results, vector_unique, k=60)
final_results = rrf_combined[:10]

# Display results
print("HYBRID RETRIEVAL: RRF RANKING")
print("=" * 80)
print(f"Query: {query}")
print(f"Total combined (RRF): {len(rrf_combined)} results")
print(f"Final ranking: Top 10 results")
print("=" * 80)

for i, result in enumerate(final_results, 1):
    source = result.get('source_method', 'Unknown')
    rank_info = ""
    
    if source == 'BM25':
        rank_info = f"(BM25 Rank #{result.get('original_rank_bm25')})"
    else:
        rank_info = f"(Vector Rank #{result.get('original_rank_vector')})"
    
    print(f"\n{i}. [{source} {rank_info}]")
    print(f"   RRF Score: {result['rrf_score']:.6f}")
    print(f"   Page: {result['page_number']}")
    print(f"   Type: {result['chunk_type']}")
    if result['breadcrumb']:
        print(f"   Breadcrumb: {result['breadcrumb']}")
    
    content_preview = result['content'][:250]
    if len(result['content']) > 250:
        content_preview += "..."
    print(f"   Content: {content_preview}")

print("\n" + "=" * 80)


HYBRID RETRIEVAL: RRF RANKING
Query: What are the fundamental rights and freedoms?
Total combined (RRF): 38 results
Final ranking: Top 10 results

1. [BM25 (BM25 Rank #1)]
   RRF Score: 0.016393
   Page: 9
   Type: section
   Breadcrumb: Part 3: Fundamental Rights and Duties > Article 17: Right to Freedom
   Content: (4) Nothing in section (d) shall be deemed to prevent the making of an Act to impose reasonable restrictions on any act which may undermi ne the nationality, sovereignty, indivisibility or Nepal, or an act of espionage against the nation, or disclosi...

2. [Vector (Vector Rank #1)]
   RRF Score: 0.016393
   Page: 18
   Type: section
   Breadcrumb: Part 3 Fundamental Rights and Duties
   Content: (4) Each peasant shall have the right to access to land as provided for in law for agricultural purposes, a long with the right to choose and preserve traditionally adopted and used endemic seeds and agricultural species.(5) The families of martyrs w...

3. [BM25 (BM25 Rank #2)]
 

In [10]:
from sentence_transformers import CrossEncoder

test_query = "What are fundamental rights?"
test_docs = [
    "Fundamental rights are basic human rights.",
    "The weather is sunny today.",
    "Right to live with dignity is a fundamental right."
]

ce_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")
test_pairs = [[test_query, doc] for doc in test_docs]
test_scores = ce_model.predict(test_pairs)

for doc, score in zip(test_docs, test_scores):
    print(f"Score: {score:.4f} | Doc: {doc}")

Score: 9.6313 | Doc: Fundamental rights are basic human rights.
Score: -11.3134 | Doc: The weather is sunny today.
Score: 6.1241 | Doc: Right to live with dignity is a fundamental right.
